## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

from pprint import pprint

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
0,0,Jiuquan,CN,39.7432,98.5174,61.39,26,0,17.90,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,56.89,86,44,2.71,scattered clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,50.11,71,75,13.80,light rain
3,3,Albany,US,42.6001,-73.9662,71.92,62,2,6.71,clear sky
4,4,Mataura,NZ,-46.1927,168.8643,49.48,93,52,9.69,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temp you would like for your trip?"))
max_temp = float(input("What is the max temp you would like for your trip?"))

What is the min temp you would like for your trip?70
What is the max temp you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp)& (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
3,3,Albany,US,42.6001,-73.9662,71.92,62,2,6.71,clear sky
7,7,Manicore,BR,-5.8092,-61.3003,76.23,88,91,4.27,overcast clouds
9,9,Bandarbeyla,SO,9.4942,50.8122,80.98,63,100,23.44,light rain
14,14,Plainview,US,40.7765,-73.4673,75.54,58,75,9.22,broken clouds
17,17,Rikitea,PF,-23.1203,-134.9692,73.83,81,100,23.09,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID         0
City            0
Country         3
Lat             0
Lng             0
Max Temp        0
Humidity        0
Cloudiness      0
Wind Speed      0
Weather Desc    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID         318
City            318
Country         318
Lat             318
Lng             318
Max Temp        318
Humidity        318
Cloudiness      318
Wind Speed      318
Weather Desc    318
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Desc", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Desc,Lat,Lng,Hotel Name
3,Albany,US,71.92,clear sky,42.6001,-73.9662,
7,Manicore,BR,76.23,overcast clouds,-5.8092,-61.3003,
9,Bandarbeyla,SO,80.98,light rain,9.4942,50.8122,
14,Plainview,US,75.54,broken clouds,40.7765,-73.4673,
17,Rikitea,PF,73.83,overcast clouds,-23.1203,-134.9692,
18,Saint-Philippe,RE,71.15,scattered clouds,-21.3585,55.7679,
20,Suez,EG,70.56,clear sky,29.9737,32.5263,
22,Ponta Do Sol,PT,70.72,few clouds,32.6667,-17.1000,
23,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
24,Atuona,PF,77.31,broken clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]= f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"]= hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
     

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np 
hotel_df["Hotel Name"].replace("", np.nan, inplace= True)
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head()

,City,Country,Max Temp,Weather Desc,Lat,Lng,Hotel Name
7,Manicore,BR,76.23,overcast clouds,-5.8092,-61.3003,Hotel Durval
9,Bandarbeyla,SO,80.98,light rain,9.4942,50.8122,JABIR HOTEL
14,Plainview,US,75.54,broken clouds,40.7765,-73.4673,Four Points by Sheraton Melville Long Island
17,Rikitea,PF,73.83,overcast clouds,-23.1203,-134.9692,People ThankYou
18,Saint-Philippe,RE,71.15,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))